*Accompanying code examples of the book "Introduction to Artificial Neural Networks and Deep Learning: A Practical Guide with Applications in Python" by [Sebastian Raschka](https://sebastianraschka.com). All code examples are released under the [MIT license](https://github.com/rasbt/deep-learning-book/blob/master/LICENSE). If you find this content useful, please consider supporting the work by buying a [copy of the book](https://leanpub.com/ann-and-deeplearning).*
  
Other code examples and content are available on [GitHub](https://github.com/rasbt/deep-learning-book). The PDF and ebook versions of the book are available through [Leanpub](https://leanpub.com/ann-and-deeplearning).

In [1]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -v -p torch

Sebastian Raschka 

CPython 3.6.4
IPython 6.2.1

torch 0.4.0


- Runs on CPU or GPU (if available)

# Model Zoo -- Multilayer Perceptron

## Imports

In [1]:
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch

## Settings and Dataset

In [2]:
##########################
### SETTINGS
##########################

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Hyperparameters
random_seed = 1
learning_rate = 0.1
num_epochs = 10
batch_size = 64

# Architecture
num_features = 784
num_hidden_1 = 128
num_hidden_2 = 256
num_classes = 10


##########################
### MNIST DATASET
##########################

# Note transforms.ToTensor() scales input images
# to 0-1 range
train_dataset = datasets.MNIST(root='data', 
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='data', 
                              train=False, 
                              transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False)

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([64, 1, 28, 28])
Image label dimensions: torch.Size([64])


In [3]:
##########################
### MODEL
##########################

class MultilayerPerceptron(torch.nn.Module):

    def __init__(self, num_features, num_classes):
        super(MultilayerPerceptron, self).__init__()
        
        ### 1st hidden layer
        self.linear_1 = torch.nn.Linear(num_features, num_hidden_1)
        # The following to lones are not necessary, 
        # but used here to demonstrate how to access the weights
        # and use a different weight initialization.
        # By default, PyTorch uses Xavier/Glorot initialization, which
        # should usually be preferred.
        self.linear_1.weight.detach().normal_(0.0, 0.1)
        self.linear_1.bias.detach().zero_()
        
        ### 2nd hidden layer
        self.linear_2 = torch.nn.Linear(num_hidden_1, num_hidden_2)
        self.linear_2.weight.detach().normal_(0.0, 0.1)
        self.linear_2.bias.detach().zero_()
        
        ### Output layer
        self.linear_out = torch.nn.Linear(num_hidden_2, num_classes)
        self.linear_out.weight.detach().normal_(0.0, 0.1)
        self.linear_out.bias.detach().zero_()
        
    def forward(self, x):
        out = self.linear_1(x)
        out = F.relu(out)
        out = self.linear_2(out)
        out = F.relu(out)
        logits = self.linear_out(out)
        probas = F.log_softmax(logits, dim=1)
        return probas

    
torch.manual_seed(random_seed)
model = MultilayerPerceptron(num_features=num_features,
                             num_classes=num_classes)

model = model.to(device)

##########################
### COST AND OPTIMIZER
##########################

cost_fn = torch.nn.NLLLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [4]:
def compute_accuracy(net, data_loader):
    net.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.view(-1, 28*28).to(device)
            targets = targets.to(device)
            probas = net(features)
            _, predicted_labels = torch.max(probas, 1)
            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
        return correct_pred.float()/num_examples * 100
    

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.view(-1, 28*28).to(device)
        targets = targets.to(device)
            
        ### FORWARD AND BACK PROP
        probas = model(features)
        cost = cost_fn(probas, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_dataset)//batch_size, cost))

    print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
          epoch+1, num_epochs, 
          compute_accuracy(model, train_loader)))

Epoch: 001/010 | Batch 000/937 | Cost: 2.4231
Epoch: 001/010 | Batch 050/937 | Cost: 0.7213
Epoch: 001/010 | Batch 100/937 | Cost: 0.3137
Epoch: 001/010 | Batch 150/937 | Cost: 0.4774
Epoch: 001/010 | Batch 200/937 | Cost: 0.3311
Epoch: 001/010 | Batch 250/937 | Cost: 0.3081
Epoch: 001/010 | Batch 300/937 | Cost: 0.3944
Epoch: 001/010 | Batch 350/937 | Cost: 0.2292
Epoch: 001/010 | Batch 400/937 | Cost: 0.0833
Epoch: 001/010 | Batch 450/937 | Cost: 0.2239
Epoch: 001/010 | Batch 500/937 | Cost: 0.2479
Epoch: 001/010 | Batch 550/937 | Cost: 0.1533
Epoch: 001/010 | Batch 600/937 | Cost: 0.2399
Epoch: 001/010 | Batch 650/937 | Cost: 0.4150
Epoch: 001/010 | Batch 700/937 | Cost: 0.2936
Epoch: 001/010 | Batch 750/937 | Cost: 0.3507
Epoch: 001/010 | Batch 800/937 | Cost: 0.1382
Epoch: 001/010 | Batch 850/937 | Cost: 0.0668
Epoch: 001/010 | Batch 900/937 | Cost: 0.2764
Epoch: 001/010 training accuracy: 94.85%
Epoch: 002/010 | Batch 000/937 | Cost: 0.3314
Epoch: 002/010 | Batch 050/937 | Cost: 

Epoch: 009/010 | Batch 900/937 | Cost: 0.0039
Epoch: 009/010 training accuracy: 99.30%
Epoch: 010/010 | Batch 000/937 | Cost: 0.0206
Epoch: 010/010 | Batch 050/937 | Cost: 0.0168
Epoch: 010/010 | Batch 100/937 | Cost: 0.0136
Epoch: 010/010 | Batch 150/937 | Cost: 0.0110
Epoch: 010/010 | Batch 200/937 | Cost: 0.0387
Epoch: 010/010 | Batch 250/937 | Cost: 0.0196
Epoch: 010/010 | Batch 300/937 | Cost: 0.0142
Epoch: 010/010 | Batch 350/937 | Cost: 0.0181
Epoch: 010/010 | Batch 400/937 | Cost: 0.0015
Epoch: 010/010 | Batch 450/937 | Cost: 0.0088
Epoch: 010/010 | Batch 500/937 | Cost: 0.0846
Epoch: 010/010 | Batch 550/937 | Cost: 0.0126
Epoch: 010/010 | Batch 600/937 | Cost: 0.0307
Epoch: 010/010 | Batch 650/937 | Cost: 0.0440
Epoch: 010/010 | Batch 700/937 | Cost: 0.0171
Epoch: 010/010 | Batch 750/937 | Cost: 0.0097
Epoch: 010/010 | Batch 800/937 | Cost: 0.0092
Epoch: 010/010 | Batch 850/937 | Cost: 0.0103
Epoch: 010/010 | Batch 900/937 | Cost: 0.0383
Epoch: 010/010 training accuracy: 99.03

In [5]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 97.55%
